In [ ]:
!pip install transformers[torch] datasets seqeval #torch accelerate loralib peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.

In [ ]:
import torch
#from torch import nn
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorForTokenClassification, AutoModelForTokenClassification
from datasets import load_from_disk, load_metric
import numpy as np
#from peft import LoraConfig, get_peft_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = load_from_disk("drive/MyDrive/ner_dataset_conll_format_augmented")
label_list = dataset["train"].features["ner_tags"].feature.names

In [ ]:
# import huggingface_hub
# huggingface_hub.login()

In [ ]:
model_name ="google-t5/t5-large" #"openai-community/gpt2-large"#"meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)#,add_prefix_space=True)
#base_model = AutoModelForTokenClassification.from_pretrained(model_name,num_labels=5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# def apply_lora(model, lora_rank=8, lora_alpha=32, lora_dropout=0.1):
#     config = LoraConfig(
#         r=lora_rank,
#         lora_alpha=lora_alpha,  # Update with correct modules
#         lora_dropout=lora_dropout,
#         bias="none",
#         task_type="TOKEN_CLASSIFICATION",
#     )
#     return get_peft_model(model, config)

In [ ]:
# class LLaMATokenClassificationModel(nn.Module):
#     def __init__(self, base_model, num_labels):
#         super(LLaMATokenClassificationModel, self).__init__()
#         self.base_model = base_model
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(self.base_model.config.d_model, num_labels)
#         self.base_model = apply_lora(self.base_model)  # Apply LoRA

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.base_model(input_ids, attention_mask=attention_mask)
#         sequence_output = self.dropout(outputs[0])
#         logits = self.classifier(sequence_output)

#         # Debugging: Print shapes
#         print(f"Shape of sequence_output: {sequence_output.shape}")  # Should be [batch_size, seq_length, hidden_size]
#         print(f"Shape of self.classifier.weight: {self.classifier.weight.shape}")  # Should be [num_labels, hidden_size]


#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             active_loss = attention_mask.view(-1) == 1
#             active_logits = logits.view(-1, self.classifier.out_features)
#             active_labels = torch.where(
#                 active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
#             )
#             loss = loss_fct(active_logits, active_labels)
#         return (loss, logits) if loss is not None else logits


In [ ]:
# #num_labels = len(dataset["train"].features["ner_tags"].feature.names)
# num_labels = 6
# #model = LLaMATokenClassificationModel(base_model, num_labels)
# model = apply_lora(base_model)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
metric = load_metric("seqeval",trust_remote_code=True)


<ipython-input-10-a617e2bdffca>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval",trust_remote_code=True)


In [ ]:
from seqeval.metrics import f1_score
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "eval_f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
training_args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=2,  # Limit the total amount of checkpoints
    greater_is_better=True,
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA"] = "1"

In [ ]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.549400,0.481223,0.585758,0.508907,0.689948,0.828034
2,0.278600,0.548108,0.653970,0.593301,0.728460,0.844161
3,0.159600,0.737910,0.731722,0.667204,0.810052,0.862085
4,0.139600,0.679196,0.761121,0.713714,0.815274,0.863454
5,0.106100,0.846957,0.751747,0.703240,0.807441,0.863026
6,0.087600,0.917226,0.770526,0.714445,0.836162,0.857978
7,0.061700,1.138706,0.775222,0.731058,0.825065,0.867190
8,0.048700,1.191678,0.784035,0.745294,0.827023,0.866177
9,0.050400,1.108227,0.770699,0.729179,0.817232,0.862270
10,0.038900,1.287941,0.767802,0.730271,0.809399,0.859889


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/se

KeyboardInterrupt: 

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_f1': 0.8098089535215285,
 'eval_loss': 0.5259455442428589,
 'eval_precision': 0.7626208378088077,
 'eval_recall': 0.8632218844984803,
 'eval_accuracy': 0.8855099849104825,
 'eval_runtime': 9.0507,
 'eval_samples_per_second': 27.622,
 'eval_steps_per_second': 3.536,
 'epoch': 20.0}

In [ ]:
trainer.predict(tokenized_datasets["test"]).predictions.shape

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IMPRESSION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(206, 587, 6)

In [ ]:
model.save_pretrained("ner_model_t5base_0835")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
"/content/drive/MyDrive/"

In [ ]:
base_model.save_pretrained(f"/content/drive/MyDrive/ner_model_google-t5_t5-large_impression_yok")
#tokenizer.save_pretrained("tokenizer_"+model_name)
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
import json
config = json.load(open(f"/content/drive/MyDrive/ner_model_google-t5_t5-large_impression_yok/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open(f"/content/drive/MyDrive/ner_model_google-t5_t5-large_impression_yok/config.json","w"))

In [ ]:
loaded = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_model_google-t5_t5-large_impression_yok")

In [ ]:
t = Trainer(
    model=loaded,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
t.evaluate(tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_f1': 0.8098089535215285,
 'eval_loss': 0.5259455442428589,
 'eval_precision': 0.7626208378088077,
 'eval_recall': 0.8632218844984803,
 'eval_accuracy': 0.8855099849104825,
 'eval_runtime': 14.2506,
 'eval_samples_per_second': 17.543,
 'eval_steps_per_second': 2.246}

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
t = Trainer(
    model=loaded,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)